In [ ]:
#!/usr/bin/env python3
import os, sys, csv
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt

In [ ]:
#Tell the user about the program
print("""
This program helps users fit an exponential function to a dataset. After providing 
the program with your dataset, and an initial guess for the rate, it will provide 
several outputs. These include a graph of your data, the fitted rate, and the goodness
of that fit.
""")

#Have the user select the form of the exponential function that works best with their data
model_type = int(input("""
Would you like model 0, 1 or 2? 
Model 0: e^(-kt)          (Best for data normalized between 1 and 0)
Model 1: e^(-kt) + C      (Best for data normalized to a maximum of 1, but that does not end near 0)
Model 2: a*e^(-kt) + C   (Best for data beginning and ending at any value. This model is the most difficult to fit)
"""))

while model_type not in [0,1,2]:
    model_type = int(input("Only 0,1 or 2 are acceptable answers. Please re-enter your selection."))
    continue
    
print(f"Proceeding with model type {model_type}.") 

In [ ]:
#Collect data from the user
print("""
Your data must be in as a two-column csv file with no headers. Column one must indicate
time, and column two must indicate the value at that timepoint. Each point must have a 
corresponding value.
""")

filepath = input("Please specify the filepath to your csv file: ")
print(f"You entered {filepath}")

with open(filepath) as csvfile:
        data = np.loadtxt(csvfile, delimiter=",", encoding="utf-8-sig")
        
myT = data[:,0]
myY = data[:,1]

In [ ]:
#Defines the exponential (growth or decay) function
def model_Exp_0(t, k):
    return np.exp(-k * t)

def model_Exp_1(t, k, C):
    return np.exp(-k * t)+C

def model_Exp_2(t, k, C, a):
    return a*np.exp(-k * t)+C


#initial guess for k
Ko = float(input("""Please provide an initial guess for your exponential rate (k). The software uses it to begin fitting the function. 0 is often a fair starting point. """))
print(f"k = {Ko}.")

if model_type in [1,2]:
    #initial guess for C
    Co = input("""You may provide an initial guess for your constant offset (C), otherwise the last entry in your data will be used.""")
    if Co == '': Co = myY[-1]
    Co = float(Co)
    print(f"C = {Co}.") 

if model_type == 2:
    #initial guess for A
    Ao = input("""You may provide an initial guess for your amplitude (a), otherwise the first entry in your data will be used.""")
    if Ao == '': Ao = myY[0]
    Ao = float(Ao)
    print(f"A = {Ao}.")

In [ ]:
if model_type == 0:
    #perform the fit
    params, cv = scipy.optimize.curve_fit(model_Exp_0, myT, myY, Ko)
    sampleRate = 20000 #per second
    tauSec = (1/params[0]) / sampleRate

    # determine quality of the fit
    squaredDiffs = np.square(myY - model_Exp_0(myT, params[0]))
    squaredDiffsFromMean = np.square(myY - np.mean(myY))
    rSquared = 1 - np.sum(squaredDiffs) / np.sum(squaredDiffsFromMean)

    # inspect the parameters
    print(f"""f(t) = e^(-{params[0]} * t)
    where k = {params[0]}""") 
    print(f"Tau = {tauSec * 1e6} µs")

elif model_type == 1:
    #perform the fit
    params, cv = scipy.optimize.curve_fit(model_Exp_1, myT, myY, [Ko, Co])
    sampleRate = 20000 #per second
    tauSec = (1/params[0]) / sampleRate    
    
    # determine quality of the fit
    squaredDiffs = np.square(myY - model_Exp_1(myT, params[0], params[1]))
    squaredDiffsFromMean = np.square(myY - np.mean(myY))
    rSquared = 1 - np.sum(squaredDiffs) / np.sum(squaredDiffsFromMean)

    # inspect the parameters
    print(f"f(t) = e^(-{params[0]} * t) + {params[1]}")
    print(f"Tau = {tauSec * 1e6} µs")
    
elif model_type == 2:
    #perform the fit
    params, cv = scipy.optimize.curve_fit(model_Exp_2, myT, myY, [Ko, Co, Ao])
    sampleRate = 20000 #per second
    tauSec = (1/params[0]) / sampleRate    
    
    # determine quality of the fit
    squaredDiffs = np.square(myY - model_Exp_2(myT, params[0], params[1], params[2]))
    squaredDiffsFromMean = np.square(myY - np.mean(myY))
    rSquared = 1 - np.sum(squaredDiffs) / np.sum(squaredDiffsFromMean)

    # inspect the parameters
    print(f"""f(t) = {params[2]}*e^(-{params[0]} * t) + {params[1]}
    where a = {params[2]}, k = {params[0]} and C = {params[1]}""") 
    print(f"Tau = {tauSec * 1e6} µs")
    
print("The fit has the following properties: ")
print(f"R² = {rSquared}")

In [ ]:
#Get save details from the user
savepath = input("If you would like to save your figure, please specify a location, file name and file type via extension. (Example: C:\folder\file.extension). Otherwise, just press enter:")
if savepath !='':
    print(f"You entered {savepath}")
else:
    print("Your figure will not be saved.")
    

In [ ]:
#Generate values for the fitted exponential function
xs2 = np.linspace(min(myT), 1.1*max(myT), 200)
if model_type == 0: 
    ys2 = model_Exp_0(xs2, params[0])
    text1 = "f(t) = e^(-{:.2f} * t)".format(params[0])
elif model_type == 1: 
    ys2 = model_Exp_1(xs2, params[0], params[1])
    text1 = "f(t) = e^(-{:.2f} * t)+{:.2f}".format(params[0],params[1])
elif model_type == 2: 
    ys2 = model_Exp_2(xs2, params[0], params[1], params[2])
    text1 = "f(t) = {:.2f} * e^(-{:.2f} * t)+{:.2f}".format(params[2],params[0],params[1])
    
#Plot the figure of your data and the fitted function
fig = plt.figure(figsize=(4,3))
plt.plot(myT, myY, '.', label="data")
plt.plot(xs2, ys2, '--', label="fitted")

#Add labels
plt.title("Fitted exponential function")
plt.xlabel("Time")
plt.ylabel("Substrate fraction")

#Overlay the function and R2 values on the figure
text2 = "R² = {:.3f}".format(rSquared)
plt.text(2.5, 0.9, text1, verticalalignment='top')
plt.text(2.5,0.8,text2, verticalalignment='top')

#Save the figure if the user specified a savepath
if savepath !='': fig.savefig(savepath, bbox_inches='tight')